In [1]:
import sidewalkify as swky
import osmnx as ox
import numpy as np
import pandas as pd
import networkx as nx
import shapely as shapely
import geopandas as gpd

In [2]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

<ipython-input-2-7721cd5e3608>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook
/home/adam/.local/lib/python3.8/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Get the initial road map
Here's what osmnx thinks of as "walking":

In [3]:
ox.downloader._get_osm_filter("walk")

'["area"!~"yes"]["highway"!~"cycleway|motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["service"!~"private"]["access"!~"private"]'

We need to separate this further into ways which could have sidewalks.

In [4]:
G = ox.graph.graph_from_place("Boston, MA", network_type="walk")

In [6]:
# At some point should figure out if this needs to be projected
gdf = ox.utils_graph.graph_to_gdfs(G)
streets = gdf[1]
streets.columns

Index(['u', 'v', 'key', 'osmid', 'oneway', 'lanes', 'name', 'highway', 'width',
       'length', 'geometry', 'maxspeed', 'ref', 'tunnel', 'service',
       'junction', 'bridge', 'access', 'area'],
      dtype='object')

In [7]:
points = gdf[0].to_file("boston_points")

In [10]:
gdf[0]

,y,x,osmid,highway,ref,geometry
61341696,42.351682,-71.068411,61341696,NaN,NaN,POINT (-71.06841 42.35168)
61341697,42.350308,-71.133251,61341697,NaN,NaN,POINT (-71.13325 42.35031)
5264310273,42.353556,-71.079073,5264310273,crossing,NaN,POINT (-71.07907 42.35356)
61341700,42.352396,-71.154430,61341700,traffic_signals,NaN,POINT (-71.15443 42.35240)
4934467588,42.269249,-71.084789,4934467588,NaN,NaN,POINT (-71.08479 42.26925)
...,...,...,...,...,...,...
61341685,42.310794,-71.050160,61341685,NaN,NaN,POINT (-71.05016 42.31079)
382402555,42.365667,-71.121712,382402555,NaN,NaN,POINT (-71.12171 42.36567)
61341692,42.352114,-71.164719,61341692,NaN,NaN,POINT (-71.16472 42.35211)
61341693,42.352406,-71.163179,61341693,NaN,NaN,POINT (-71.16318 42.35241)


In [ ]:

print(sum(streets.highway.value_counts())/len(streets.highway))

In [ ]:
shvcs = streets.highway.value_counts().index
highway_types = {*([x for x in shvcs if type(x)==str] + [x for y in shvcs for x in y if type(y)==list])}
highway_types

In [ ]:
G_highway_types={}
for htype in highway_types:
    cf = '["highway"~"{}"]'.format(htype)
    G_highway_types[htype] = ox.graph.graph_from_place("Boston, MA", network_type="walk", custom_filter=cf)


### Footway

In [ ]:
ox.plot_graph_folium(G_highway_types['footway'])

I feel like we can remove all of these EXCEPT the Common!  Ignore for now.

### Corridor

In [ ]:
ox.plot_graph_folium(G_highway_types['corridor'])

Think we can safely leave it out for now...

### Crossing

In [ ]:
ox.plot_graph_folium(G_highway_types['crossing'])

no

### Living Street

In [ ]:
ox.plot_graph_folium(G_highway_types['living_street'])

### Path

In [ ]:
ox.plot_graph_folium(G_highway_types['path'])

add later

### pedestrian


In [ ]:
ox.plot_graph_folium(G_highway_types['pedestrian'])

later

### primary, primary_link

In [ ]:
ox.plot_graph_folium(G_highway_types['primary'])

In [ ]:
ox.plot_graph_folium(G_highway_types['primary_link'])

of course keep but interesting!

### residential

In [ ]:
ox.plot_graph_folium(G_highway_types['residential'])

### road


In [ ]:
ox.plot_graph_folium(G_highway_types['road'])

??? check by hand?

### secondary 


In [ ]:
ox.plot_graph_folium(G_highway_types['secondary'])

In [ ]:
ox.plot_graph_folium(G_highway_types['secondary_link'])

don't include because the paths are elsewhere?

at least don't include for now

### service

In [ ]:
ox.plot_graph_folium(G_highway_types['service'])

either these are walkable or not -- check!

### steps

In [ ]:
ox.plot_graph_folium(G_highway_types['steps'])

uh

### tertiary

In [ ]:
ox.plot_graph_folium(G_highway_types['tertiary'])

In [ ]:
ox.plot_graph_folium(G_highway_types['tertiary_link'])

### track
 

In [ ]:
ox.plot_graph_folium(G_highway_types['track'])

??? check by hand

### trunk

In [ ]:
ox.plot_graph_folium(G_highway_types['trunk'])

think we have to include these

### trunk_link

In [ ]:
ox.plot_graph_folium(G_highway_types['trunk_link'])

include but look by hand

### Unclassified

In [ ]:
ox.plot_graph_folium(G_highway_types['unclassified'])

ignore for now

## summary

For now, include:
* trunk_link
* trunk
* tertiary
* tertiary_link
* secondary
* residential
* primary
* primary_link



In [5]:
cf = '["highway"~"trunk|trunk_link|tertiary|tertiary_link|secondary|residential|primary|primary_link"]'
G_roads_init = ox.graph.graph_from_place("Boston, MA", network_type="walk", custom_filter=cf)
